## Reading Vector Files using Fiona

https://github.com/Toblerity/Fiona


Fiona streams simple feature data to and from GIS formats like GeoPackage and Shapefile.

Fiona can read and write real-world data using multi-layered GIS formats, zipped and in-memory virtual file systems, from files on your hard drive or in cloud storage. This project includes Python modules and a command line interface (CLI).

Fiona depends on GDAL but is different from GDAL's own bindings. Fiona is designed to be highly productive and to make it easy to write code which is easy to read.

We're using the GeoPackage file format. To list all available layers:

## Data

Let's download some data we need later. 

**Natural Earth** is a public domain map dataset available at 1:10m, 1:50m, and 1:110 million scales. Featuring tightly integrated vector and raster data, with Natural Earth you can make a variety of visually pleasing, well-crafted maps with cartography or GIS software. More info: https://www.naturalearthdata.com/

In [5]:
import geoutils
import zipfile
import os

if not os.path.exists("geodata"):
    os.mkdir("geodata")
    geoutils.download(geoutils.geodata["natural-earth"], "geodata/ne.gpkg.zip")
    with zipfile.ZipFile("geodata/ne.gpkg.zip", 'r') as z:
        z.extractall("geodata")

In [6]:
import fiona

fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'raw',
 'OGR_PDS': 'r',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [3]:
if 'GPKG' in fiona.supported_drivers:
    print("GeoPackage is supported...")

GeoPackage is supported...


In [4]:
import fiona

c = fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports')

airport = next(iter(c))
airport

DriverError: geodata/packages/natural_earth_vector.gpkg: No such file or directory

In [ ]:
airport['properties']['name']

In [ ]:
airport['geometry']['type']

In [ ]:
airport['geometry']['coordinates']

In [ ]:
c.close()

Lets read all data. There are basically two ways:

a) load everything to memory: (if dataset isn't too large...)

    alldata = list(c)
    
b) iterate through all data: (one by one):

    for element in c:
        ...

In [ ]:
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports') as c:
    for airport in c:
        if airport['properties']['iata_code'] == "ZRH":
            #print(airport['properties']['name'])
            #print(airport['geometry']['coordinates'])
            #print(airport['properties']['wikipedia'])
            print(airport)

In [ ]:
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports') as c:
    print(c.crs)

In [ ]:
import fiona

c = fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_admin_0_countries')

country = next(iter(c))

print(country['properties']['NAME'])
print(country['properties']['NAME_ZH'])
print(country['properties']['CONTINENT'])
print(country['properties']['POP_EST'])
print(country['properties']['POP_YEAR'])


In [ ]:


with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_admin_0_countries') as c:
    for country in c:
        if country['properties']['NAME'] == "Switzerland":
            print(country['properties']['POP_EST'])
            print(country['properties']['POP_YEAR'])   
            print(country['geometry']['type'])
            # print(country['geometry']['coordinates']) # you don't want to print all coordinates!
            geomtype = country['geometry']['type']
            geometry = country['geometry']                 

## Find Airports inside the polygon

First we convert it to a shapely Polygon. If you use another country, it may be a Multipolygon, check the output above!

In [ ]:
from shapely.geometry import shape
from shapely.geometry import Polygon

ch = Polygon(shape(geometry))  # some other countries are MultiPolygon, check the type!
ch

In [ ]:
from shapely.geometry import Point

with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_airports') as c:
    for airport in c:      
        position = Point(airport['geometry']['coordinates'])
        if position.within(ch):
            print(airport['properties']['iata_code'], airport['properties']['name'], airport['geometry']['coordinates'])

### Find Areas with a population greater than 25'000 inside the Polygon

In [ ]:
import fiona

cnt = 0
with fiona.open('geodata/packages/natural_earth_vector.gpkg', 'r', layer='ne_10m_populated_places') as c:
    for place in c:
        geom = place["geometry"]
        position = Point(place['geometry']['coordinates'])
        name = place["properties"]["NAME"]
        pop = int(place["properties"]["POP_MAX"])
        cnt+=1
        if pop>25000 and position.within(ch):
            print(name, pop, position)
            
print(cnt)